In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from recomm.features import index_single_feature
from recomm.classifier import ClassifierNN, ClassifierDNN

In [2]:
init_notebook_mode(connected=True)

# Introduction

In this document, we are going to implement a xor gate with neural network and to check the performance of the comparison between two arbitrary values.

# XOR Gate

In [50]:
features = np.random.rand(50000, 2)
features = pd.DataFrame(np.where(features > 0.5, 1, 0))
labels = np.where(features[0] == features[1], 0, 1)
labels = pd.get_dummies(labels)

In [51]:
features.head()

,0,1
0,0,0
1,0,0
2,0,0
3,1,0
4,1,1


In [52]:
labels.head()

,0,1
0,1,0
1,1,0
2,1,0
3,0,1
4,1,0


In [53]:
train_features = features.iloc[:40000].values
train_labels = labels.iloc[:40000].values
test_features = features.iloc[40000:].values
test_labels = labels.iloc[40000:].values

In [54]:
classifier = ClassifierNN(train_features, train_labels)\
            .build_network()\
            .set_objective(method="l2_loss")\
            .optimize()\
            .estimate(batch_size=1000, learning_rate=1e-2)\
            .predict(test_features)\
            .activate_label()\
            .get_accuracy(test_labels)

In [55]:
data = [go.Scatter(x=np.arange(1000),
                   y=np.log(np.array(classifier.loss)),
                   mode="lines",
                   name="layer_number: 10"),]
layout = go.Layout(
    title='Learning Rate: 1e-2',
    xaxis=dict(
        title='iteration steps',
    ),
    yaxis=dict(
        title='L2 Accuracy (Log Scale)'
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

The accuracy is half which is poor.

In [56]:
accuracy

0.50340000000000007

There is a lecture https://aimatters.wordpress.com/2016/01/11/solving-xor-with-a-neural-network-in-python/ which teaching us what to implement an xor gate. However, it is similar to consider (1, 1) as another label and the model is conducted after the known solution.

In [60]:
features = np.random.rand(50000, 2)
features = pd.DataFrame(np.where(features > 0.5, 1, 0))
labels = features[0] + features[1]
labels = pd.get_dummies(labels)

In [61]:
features.head()

,0,1
0,0,0
1,1,1
2,1,0
3,0,0
4,1,0


In [62]:
labels.head()

,0,1,2
0,1,0,0
1,0,0,1
2,0,1,0
3,1,0,0
4,0,1,0


In [63]:
train_features = features.iloc[:40000].values
train_labels = labels.iloc[:40000].values
test_features = features.iloc[40000:].values
test_labels = labels.iloc[40000:].values

In [64]:
classifier = ClassifierNN(train_features, train_labels)\
            .build_network()\
            .set_objective(method="l2_loss")\
            .optimize()\
            .estimate(batch_size=1000, learning_rate=1e-2)\
            .predict(test_features)\
            .activate_label()\
            .get_accuracy(test_labels)

Then, the rather high fidelity is obtained after considering (1, 1) as another label.

In [65]:
classifier.accuracy

1.0

What if we consider the 3rd label as a hidden layer, and then implement DNN to search the solution. The second layer must reduce the dimension from 3 to 2. After that, it is just necessary to recognize the result of the 1st label. For example, if the 1st exists, then another label must be 0; otherwise, 2nd label must be 1 once the 1st label is 0.

In [78]:
features = 3 * np.random.rand(50000)
features = pd.get_dummies(np.ceil(features))
labels = np.where(features[1.0] == 0, 1, 0)
labels = pd.get_dummies(labels)

In [79]:
features.head()

,1.0,2.0,3.0
0,0,0,1
1,0,0,1
2,1,0,0
3,0,1,0
4,0,0,1


In [80]:
labels.head()

,0,1
0,0,1
1,0,1
2,1,0
3,0,1
4,0,1


In [81]:
train_features = features.iloc[:40000].values
train_labels = labels.iloc[:40000].values
test_features = features.iloc[40000:].values
test_labels = labels.iloc[40000:].values
classifier = ClassifierNN(train_features, train_labels)\
            .build_network()\
            .set_objective(method="l2_loss")\
            .optimize()\
            .estimate(batch_size=1000, learning_rate=1e-2)\
            .predict(test_features)\
            .activate_label()\
            .get_accuracy(test_labels)

In [82]:
classifier.accuracy

1.0

It is found an xor gate can be implemented by two steps. The next step, we are going to exam whether DNN is able to implement a xor gate with a single layer which has 3 nodes.

In [93]:
features = np.random.rand(50000, 2)
features = pd.DataFrame(np.where(features > 0.5, 1, 0))
labels = np.where(features[0] == features[1], 0, 1)
labels = pd.get_dummies(labels)
train_features = features.iloc[:40000].values
train_labels = labels.iloc[:40000].values
test_features = features.iloc[40000:].values
test_labels = labels.iloc[40000:].values
classifier = ClassifierDNN(train_features, train_labels)\
            .build_network(hidden_layers=[3], activate=True)\
            .set_objective(method="l2_loss")\
            .optimize()\
            .estimate(batch_size=1000, learning_rate=1e-2, iter_max=400)\
            .estimate(batch_size=1000, learning_rate=1e-2, iter_max=600, init=False)\
            .predict(test_features)\
            .activate_label()\
            .get_accuracy(test_labels)

In [94]:
data = [go.Scatter(x=np.arange(1000),
                   y=np.log(np.array(classifier.loss)),
                   mode="lines",
                   name="layer_number: 10"),]
layout = go.Layout(
    title='Learning Rate: 1e-2',
    xaxis=dict(
        title='iteration steps',
    ),
    yaxis=dict(
        title='L2 Accuracy (Log Scale)'
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Finally, an xor gate is realized with DNN.

In [95]:
classifier.accuracy

1.0

# Number Comparison

In this section, we will compare two numbers to find the larger one.

In [85]:
features = np.random.rand(50000, 2)
features = pd.DataFrame(features)
labels = np.where(features[0] > features[1], 0, 1)
labels = pd.get_dummies(labels)

In [86]:
features.head()

,0,1
0,0.952175,0.764018
1,0.558042,0.547050
2,0.014297,0.788431
3,0.750238,0.782643
4,0.541670,0.095060


In [87]:
labels.head()

,0,1
0,1,0
1,1,0
2,0,1
3,0,1
4,1,0


In [88]:
train_features = features.iloc[:40000].values
train_labels = labels.iloc[:40000].values
test_features = features.iloc[40000:].values
test_labels = labels.iloc[40000:].values
classifier = ClassifierNN(train_features, train_labels)\
            .build_network()\
            .set_objective(method="l2_loss")\
            .optimize()\
            .estimate(batch_size=1000, learning_rate=1e-2)\
            .predict(test_features)\
            .activate_label()\
            .get_accuracy(test_labels)

The result is amazing. We are able to find the larger number, it is difficult to level a series of numbers.

In [89]:
classifier.accuracy

0.99129999999999996